#### Faiss
Facebook AI Similarity Search(Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning

In [2]:
from langchain_community.document_loaders import TextLoader 
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings 
from langchain_text_splitters import CharacterTextSplitter
loader=TextLoader('speech.txt')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents=documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [4]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs,embedding=embeddings)
db

/var/folders/f_/lc89msh91h903jdcn4r4lnlw0000gn/T/ipykernel_50556/3159537081.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings()


In [6]:
### querying 
query="how does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [8]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better

In [9]:
docs_and_sscore=db.similarity_search_with_score(query)

In [10]:
docs_and_sscore

[(Document(id='1f41ede1-5d4a-4617-b7e6-c4603190ed5a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(16303.627)),
 (Document(id='035db20a-d42e-438f-9e47-e3891117c107', metadata={'source': 'spe

In [12]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[2.3306267261505127,
 -3.968899965286255,
 1.1118825674057007,
 1.9423677921295166,
 -0.239581897854805,
 0.7353692650794983,
 1.6211892366409302,
 -0.792922854423523,
 0.44230005145072937,
 -1.4448784589767456,
 1.1796129941940308,
 -1.343953251838684,
 -0.4402468502521515,
 1.2353644371032715,
 -0.13697770237922668,
 -1.5735127925872803,
 -0.1286386102437973,
 0.3762131929397583,
 1.3467803001403809,
 -1.3253976106643677,
 -0.42093196511268616,
 -0.2261018306016922,
 2.5550167560577393,
 -2.3077871799468994,
 0.35487088561058044,
 -1.555479884147644,
 0.35823187232017517,
 -2.2210235595703125,
 1.4886583089828491,
 -1.0074082612991333,
 1.8543516397476196,
 -2.7853548526763916,
 -2.228325843811035,
 2.9534220695495605,
 2.752462863922119,
 -4.344803333282471,
 -0.6361454129219055,
 1.1893184185028076,
 -0.5666293501853943,
 -0.9434905648231506,
 -0.7323119640350342,
 -2.806986093521118,
 2.378882884979248,
 0.37672561407089233,
 0.6226173639297485,
 -0.0523809976875782,
 -0.372053861

In [13]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='1f41ede1-5d4a-4617-b7e6-c4603190ed5a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='035db20a-d42e-438f-9e47-e3891117c107', metadata={'source': 'speech.txt'}, page_content='To 

In [14]:
### Saving and Loading 
db.save_local('faiss_index')

In [15]:
new_df=FAISS.load_local('faiss_index',embeddings,allow_dangerous_deserialization=True)
docs=new_df.similarity_search(query)

In [16]:
docs

[Document(id='1f41ede1-5d4a-4617-b7e6-c4603190ed5a', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='035db20a-d42e-438f-9e47-e3891117c107', metadata={'source': 'speech.txt'}, page_content='To 